# Msc Thesis

Building an FR model

## LIBRARY IMPORTS

In [33]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from mtcnn.mtcnn import MTCNN
import cv2
from tqdm.notebook import tqdm

## LOAD DATASET AND PREPROCESS

A few preprocessing steps are required to make the dataset ready for training

- Extrace faces
- Replace images with the extracted faces (This is common and much more efficient than including the whole image, since it can include many unimportant features)
- Rescale the images to a smaller size (also a matter of efficiency. It is handled by the face extraction function formulated below)
- Saving the train and test sets after preprocessing as numpy objects to enable easy reproduction of experiments in the future by just loading the numpy object.
- Creating batch organised datasets


### Face Extraction

In [34]:
# can detect multiple faces but for our tasks we assume just one face images only
def extract_faces_from_img(imagePath, required_size=(160, 160)):
    image = Image.open(imagePath)
    image = np.asarray(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = faceCascade.detectMultiScale(
                                        gray,
                                        scaleFactor=1.3,
                                        minNeighbors=3,
                                        minSize=(30, 30)
                                        )
    if len(faces) != 0:
        for (x, y, w, h) in faces:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            roi_color = image[y:y + h, x:x + w]
        # resize pixels to the model size
        image = Image.fromarray(roi_color)
        image = image.resize(required_size)
        face_array = np.asarray(image)
    else:
        # if face is not detected we need to return None
        face_array = None
    return face_array

## Custom 6-person dataset for a classiffier

In [35]:
samples = {
            'n002462':'Elizabeth Olsen',
            'n002038':'David Hasselhoff',
            'n000419':'Amy Adams',
            'n000709':'Antonio Banderas',
            'n001063':'Billy Idol',
            'n003862':'Jason Derulo'
}

train_ds_path = r'C:\Users\Keravnos\Documents\VGG-Face2\data\vggface2_train.tar\train'

def prepare_dataset(samples, train_data_path):
    ds = []
    # list out dict keys
    key_list = list(samples.keys())
    #iterate over our 6 identity directories
    for k in tqdm(key_list,'Identities'):
        # use this as label
        label = samples[k]
        #directory path of images for each identity
        dir_path = os.path.join(train_data_path,k)
        print('Extracting images for', label)
        # iterate over image directory and extract faces
        for img in tqdm(os.listdir(dir_path),'Images'):
            # need the image path for the face extraction method
            img_path = os.path.join(dir_path,img)
            # create a face array variable where the extracted face is constructed by the function.
            face_array = extract_faces_from_img(img_path)
            # add both image and label in dataset
            ds.append((face_array,label))
    print('DATASET COMPILED!')
    return ds

In [36]:
ds = prepare_dataset(samples,train_ds_path)

Identities:   0%|          | 0/6 [00:00<?, ?it/s]

Extracting images for Elizabeth Olsen


Images:   0%|          | 0/559 [00:00<?, ?it/s]

Extracting images for David Hasselhoff


Images:   0%|          | 0/542 [00:00<?, ?it/s]

...

Extracting images for Billy Idol


Images:   0%|          | 0/396 [00:00<?, ?it/s]

Extracting images for Jason Derulo


Images:   0%|          | 0/508 [00:00<?, ?it/s]

DATASET COMPILED!


In [37]:
ds = np.asarray(ds, dtype=object)
ds.shape

(3145, 2)

Clean all non recognised images from dataset.

In [38]:
ds = [(image,label) for (image,label) in ds if image.__class__.__name__ != 'NoneType' ]

In [39]:
ds = np.asarray(ds, dtype=object)
ds.shape

(2599, 2)

In [40]:
from sklearn.preprocessing import LabelEncoder

images = [image for image,label in ds]
labels = [label for image,label in ds]

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = tf.keras.utils.to_categorical(encoded_Y)

SPLIT

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, dummy_y, test_size=0.3, random_state=42)

Preprocess for xception model and prepare batch size

In [42]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train,y_train))

In [43]:
test_dataset = tf.data.Dataset.from_tensor_slices((X_test,y_test))

In [44]:
def preprocess(image,label):
    image = img_final = tf.cast(image, tf.float32) / 255.0
    final_image = tf.keras.applications.xception.preprocess_input(image)
    return final_image,label

batch_size = 4
train_dataset = train_dataset.shuffle(len(train_dataset)*2)
train_dataset = train_dataset.map(preprocess).batch(batch_size).prefetch(1)

test_dataset = test_dataset.shuffle(len(test_dataset)*2)
test_dataset = test_dataset.map(preprocess).batch(batch_size).prefetch(1)


## PREPROCESS DATASET

## BUILDING THE MODEL

Use Transfer Learning to build on top of an Xception model architecture for image classification.

In [56]:
from tensorflow import keras
from tensorflow.keras import layers

n_classes = len(list(samples.keys()))

base_model = keras.applications.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(160, 160, 3),
    include_top=False
)  # Do not include the ImageNet classifier at the top.


avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
# drop = keras.layers.Dropout(0.2)(avg)  # Regularize with dropout
output = keras.layers.Dense(n_classes, activation='softmax')(avg)
model = keras.Model(base_model.input, output)

# Freeze the base_model
for layer in base_model.layers:
    layer.trainable = False


model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 79, 79, 32)   864         input_9[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 79, 79, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 79, 79, 32)   0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [58]:
from tensorflow.keras.optimizers import Adam, SGD, Nadam
from tensorflow.keras.callbacks import ModelCheckpoint
import math

initial_learning_rate = 0.2
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)

# optimizer = SGD(learning_rate=0.2)
optimizer = Adam(learning_rate=lr_schedule)
# optimizer = Nadam(0.2)

# recompile
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy'])

# train
history = model.fit(train_dataset,
                    batch_size=batch_size,
                    epochs=200,
                    validation_data=test_dataset,
                    verbose=1,
                    callbacks=[
                                ModelCheckpoint(filepath='model_ckp/model_at_ep{epoch}.h5'),
                                    ]
)
model.save('model_ckp/model_final.h5')

Epoch 1/200
455/455 [==============================] - 17s 30ms/step - loss: 1.9853 - accuracy: 0.2848 - val_loss: 1.4426 - val_accuracy: 0.4372
Epoch 2/200
455/455 [==============================] - 13s 27ms/step - loss: 1.6845 - accuracy: 0.3964 - val_loss: 1.8393 - val_accuracy: 0.3462
Epoch 3/200
455/455 [==============================] - 13s 27ms/step - loss: 1.5709 - accuracy: 0.4426 - val_loss: 1.6275 - val_accuracy: 0.3859
Epoch 4/200
455/455 [==============================] - 12s 27ms/step - loss: 1.5243 - accuracy: 0.4535 - val_loss: 2.0337 - val_accuracy: 0.3923
Epoch 5/200
455/455 [==============================] - 13s 28ms/step - loss: 1.5348 - accuracy: 0.4805 - val_loss: 1.6703 - val_accuracy: 0.4244
Epoch 6/200
455/455 [==============================] - 13s 28ms/step - loss: 1.2692 - accuracy: 0.5459 - val_loss: 1.5589 - val_accuracy: 0.4346
Epoch 7/200
455/455 [==============================] - 13s 28ms/step - loss: 1.3461 - accuracy: 0.5261 - val_loss: 1.6431 - val_ac

### Second round of training by manual train stopping

Reuse the code from above to get the model checkpoint at which validationa accuracy is at its highest and continue training until we reach the 70-80% threshold

In [60]:
reloaded_model = tf.keras.models.load_model('model_ckp/model_at_ep190.h5')


initial_learning_rate = 0.02
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.9,
    staircase=True)

optimizer = SGD(learning_rate=lr_schedule)

# recompile
reloaded_model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy'])

# train
history = reloaded_model.fit(train_dataset,
                    batch_size=batch_size,
                    epochs=50,
                    validation_data=test_dataset,
                    verbose=1,
                    callbacks=[
                                ModelCheckpoint(filepath='model_ckp/model_at_ep{epoch}_run2.h5'),
                                    ]
)
model.save('model_ckp/model_final.h5')

Epoch 1/50
455/455 [==============================] - 16s 28ms/step - loss: 0.2234 - accuracy: 0.9269 - val_loss: 1.4712 - val_accuracy: 0.6987
Epoch 2/50
455/455 [==============================] - 12s 26ms/step - loss: 0.2147 - accuracy: 0.9318 - val_loss: 1.4703 - val_accuracy: 0.6987
Epoch 3/50
455/455 [==============================] - 12s 26ms/step - loss: 0.2136 - accuracy: 0.9340 - val_loss: 1.4717 - val_accuracy: 0.6936
Epoch 4/50
455/455 [==============================] - 12s 26ms/step - loss: 0.2136 - accuracy: 0.9324 - val_loss: 1.4716 - val_accuracy: 0.6936
Epoch 5/50
455/455 [==============================] - 12s 26ms/step - loss: 0.2137 - accuracy: 0.9335 - val_loss: 1.4706 - val_accuracy: 0.6949
Epoch 6/50
455/455 [==============================] - 12s 26ms/step - loss: 0.2136 - accuracy: 0.9324 - val_loss: 1.4708 - val_accuracy: 0.6949
Epoch 7/50
109/455 [======>.......................] - ETA: 6s - loss: 0.2127 - accuracy: 0.9312

KeyboardInterrupt: 

Fine Tuning

In [183]:
reloaded_model = tf.keras.models.load_model('model_ckp/model_at_ep190.h5')
# unfreeze and retrain
for layer in base_model.layers:
    layer.trainable = True
    
optimizer = SGD(learning_rate=0.01, momentum=0.9, decay=0.001)

# recompile
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy'])

# train
history = model.fit(train_dataset,
                    batch_size=batch_size,
                    epochs=20,
                    validation_data=test_dataset,
                    verbose=1,
                    callbacks=[
                                ModelCheckpoint(filepath='model_FT_ckp/model_FT_at_ep{epoch}.h5'),
                                    ]
)
model.save('model_FT_ckp/model_FT_final.h5')

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# for x,y in test:
#     print(y)

Evaluate

In [26]:
test_path = r'C:\Users\Keravnos\Documents\VGG-Face2\data\vggface2_train.tar\train\n002038\0107_01.jpg'
load_test_face = extract_faces_from_img(test_path)
lbl = ''

print(load_test_face.shape)
img, lbl = preprocess(load_test_face,lbl)
test_img_ds = tf.data.Dataset.from_tensor_slices(img)
pred = model.predict(test_img_ds, batch_size=None)

(160, 160, 3)


ValueError: in user code:

    C:\Users\Keravnos\Documents\AdversarialFR\venv\lib\site-packages\tensorflow\python\keras\engine\training.py:1569 predict_function  *
        return step_function(self, iterator)
    C:\Users\Keravnos\Documents\AdversarialFR\venv\lib\site-packages\tensorflow\python\keras\engine\training.py:1559 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Keravnos\Documents\AdversarialFR\venv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Keravnos\Documents\AdversarialFR\venv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Keravnos\Documents\AdversarialFR\venv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kw

In [209]:
print(pred)

NameError: name 'pred' is not defined